In [1]:
import numpy as np
import pandas as pd

import ast

import re
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
raw_link = pd.read_csv('preprocess_1.csv')
raw_tthc = pd.read_csv('raw_tthc.csv', dtype={'maThuTuc': str})

In [3]:
raw_link.info()
raw_link.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9289 entries, 0 to 9288
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   link            9289 non-null   object
 1   phanLoai        9289 non-null   object
 2   boNganh         5939 non-null   object
 3   cauHoi          9289 non-null   object
 4   cauTraLoi       9289 non-null   object
 5   TTHCLienQuan    9289 non-null   object
 6   cauHoiLienQuan  9289 non-null   object
dtypes: object(7)
memory usage: 508.1+ KB


,link,phanLoai,boNganh,cauHoi,cauTraLoi,TTHCLienQuan,cauHoiLienQuan
2734,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"['Công dân', 'Doanh nghiệp', 'Tổ chức khác']",Bộ Nông nghiệp và Môi trường,Kính gửi Chi cục Quản lý chất lượng nông lâm t...,Thành phần hồ sơ thực hiện TTHC cấp lại Giấy c...,['https://dichvucong.gov.vn/p/home/dvc-chi-tie...,['https://dichvucong.gov.vn/p/home/dvc-chi-tie...


In [4]:
raw_link['cauHoiGoc'] = raw_link['cauHoi']
raw_link['cauTraLoiGoc'] = raw_link['cauTraLoi']

In [5]:
def clean_text(text):
    text = unicodedata.normalize('NFC', text)
    text = text.lower()
    # Remove characters that are not letters or spaces, including punctuation.
    cleaned_text = re.sub(r'[^a-z0-9àáạảãâầấậẩẫăắằẵẳặđèẹẻéẽêềếệểễìíịỉĩỵỳýỷỹóọỏõòôồốộổỗơợởỡớờùúụủũưừứựửữ/\s]+', ' ', text)
    # Trim extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

raw_link = raw_link.assign(cauHoi=raw_link['cauHoiGoc'].apply(clean_text))
raw_link = raw_link.assign(cauTraLoi=raw_link['cauTraLoiGoc'].apply(clean_text))

In [6]:
q = raw_link['cauHoi'].to_numpy()
a = raw_link['cauTraLoi'].to_numpy()

vectorizer = TfidfVectorizer()
threshold = 0.95

tfidf_matrix_q = vectorizer.fit_transform(q) 
tfidf_matrix_a = vectorizer.fit_transform(a) 

res = []
idx = []

for i in range(len(raw_link)):
    if i in res:
        continue

    similarities_q = cosine_similarity(tfidf_matrix_q[i], tfidf_matrix_q).flatten()
    similarities_a = cosine_similarity(tfidf_matrix_a[i], tfidf_matrix_a).flatten()

    idx_q = np.where(similarities_q >= threshold)
    idx_a = np.where(similarities_a >= threshold)

    intersection = np.intersect1d(idx_q, idx_a)
    intersection = intersection[(intersection > i) & (~np.isin(intersection, res))].tolist()

    res.extend(intersection)
    
    if len(intersection) > 0:
        intersection.extend([i])
        idx.append(intersection)

In [7]:
print(len(res), len(set(res)))
raw_link = raw_link.drop(index=res)

84 84


In [8]:
raw_link = raw_link.reset_index(drop=True)
raw_link = raw_link.drop(columns=['cauHoi', 'cauTraLoi'])
raw_link = raw_link.rename(columns={'cauHoiGoc': 'cauHoi', 'cauTraLoiGoc': 'cauTraLoi'})
raw_link = raw_link[['link', 'phanLoai', 'boNganh', 'cauHoi', 'cauTraLoi', 'TTHCLienQuan', 'cauHoiLienQuan']]

In [9]:
raw_link.info()
raw_link.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   link            9205 non-null   object
 1   phanLoai        9205 non-null   object
 2   boNganh         5885 non-null   object
 3   cauHoi          9205 non-null   object
 4   cauTraLoi       9205 non-null   object
 5   TTHCLienQuan    9205 non-null   object
 6   cauHoiLienQuan  9205 non-null   object
dtypes: object(7)
memory usage: 503.5+ KB


,link,phanLoai,boNganh,cauHoi,cauTraLoi,TTHCLienQuan,cauHoiLienQuan
1902,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,['Công dân'],Bộ Nông nghiệp và Môi trường,"Khi mang động vật, sản phẩm động vật thủy sản ...","Có, thực hiện theo quy định tại Điều 18 Thông ...",['https://dichvucong.gov.vn/p/home/dvc-chi-tie...,[]


In [10]:
raw_link.to_csv('preprocess_2.csv', index=False)